In [1]:
#!pip install pandas
#!pip install openmeteo-requests
#!pip install requests-cache
#!pip install retry-requests
#!pip install kafka

In [2]:
#imports
import openmeteo_requests
import requests_cache
import pandas as pd
from retry_requests import retry
from kafka import KafkaProducer
import json

In [3]:
#set parameters
source_url = "https://archive-api.open-meteo.com/v1/archive"
kafka_server = 'localhost:9092'
api_params = {
	"latitude": 48.210033,
	"longitude": 16.363449,
	"start_date": "2020-01-01",
	"end_date": "2024-01-11",
	"hourly": ["temperature_2m", "relative_humidity_2m", "wind_speed_10m"]
}

In [4]:
#set up api
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)


# The order of variables in hourly or daily is important to assign them correctly b

#get api data and parse into variables
responses = openmeteo.weather_api(source_url, params=api_params)
response = responses[0]

hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
hourly_relative_humidity_2m = hourly.Variables(1).ValuesAsNumpy()
hourly_wind_speed_10m = hourly.Variables(2).ValuesAsNumpy()
#create dataframe
hourly_data = {"time": pd.date_range(
start = pd.to_datetime(hourly.Time(), unit = "s"),
end = pd.to_datetime(hourly.TimeEnd(), unit = "s"),
freq = pd.Timedelta(seconds = hourly.Interval()),
inclusive = "left"
)}
hourly_data["temperature_2m"] = hourly_temperature_2m
hourly_data["relative_humidity_2m"] = hourly_relative_humidity_2m
hourly_data["wind_speed_10m"] = hourly_wind_speed_10m
hourly_dataframe = pd.DataFrame(data = hourly_data)

In [5]:
#init producer
producer = KafkaProducer(bootstrap_servers=kafka_server,value_serializer=lambda v: json.dumps(v).encode('utf-8'))

In [6]:
#produce df to kafka row by row
for index, row in hourly_dataframe.iterrows():
    producer.send('weather', row.to_dict())
    producer.flush()

In [7]:
producer.close()